# Connectivity Manager

In version 1.9.3, `ConnMan` is introduced to manage the bus related connectivity changes.

It is an attribute `conn` of the `System` instance.

In [1]:
import andes

In [2]:
andes.config_logger(stream_level=20)

## Scenario 1

For buses that are turned off by case file, the connectivity is updated in the `System.setup()`.

We can see that the connected devices to off-line buses are turned off.

In [3]:
ss = andes.load(andes.get_case('ieee14/ieee14_conn.xlsx'),
                setup=True, no_output=True, default_config=True)

Working directory: "/Users/jinningwang/work/andes/examples/demonstration"
> Loaded generated Python code in "/Users/jinningwang/.andes/pycode".
Generated code for <PQ> is stale.
Numerical code generation (rapid incremental mode) started...


Generating code for 1 models on 12 processes.


Saved generated pycode to "/Users/jinningwang/.andes/pycode"
> Reloaded generated Python code of module "pycode".
Generated numerical code for 1 models in 0.1019 seconds.
Parsing input file "/Users/jinningwang/work/andes/andes/cases/ieee14/ieee14_conn.xlsx"...
Input file parsed in 0.2000 seconds.
Entering connectivity update.
Following bus(es) are turned off: [15]
-> System connectivity update results:
In <ACLine>, turn off ['Line_21']
In <StaticGen>, turn off [6]
In <StaticLoad>, turn off ['PQ_12']
In <StaticShunt>, turn off ['Shunt_3']
-> System connectivity check results:
  1 islanded bus detected.
  System is interconnected.
  Each island has a slack bus correctly defined and enabled.
System internal structure set up in 0.0207 seconds.


In [4]:
_ = ss.PFlow.run()

-> System connectivity check results:
  1 islanded bus detected.
  System is interconnected.
  Each island has a slack bus correctly defined and enabled.

-> Power flow calculation
           Numba: Off
   Sparse solver: KLU
 Solution method: NR method
Power flow initialized in 0.0109 seconds.
0: |F(x)| = 0.5605182134
1: |F(x)| = 0.006202200332
2: |F(x)| = 5.819382825e-06
3: |F(x)| = 6.964193111e-12
Converged in 4 iterations in 0.0106 seconds.


As expected, there is no power injection from the turned off devices.

Note: `Line.a1.e` is the active power injection at the from_bus (`bus1`).

In [5]:
ss.Line.get(src='a1', attr='e', idx=['Line_21'])

array([0.])

`PV.a.e` is the active power injection at the connected bus.

In [6]:
ss.PV.get(src='a', attr='e', idx=[6])

array([-0.])

Similarly, `PQ.a.e` is the active power injection at the connected bus.

In [7]:
ss.PQ.get(src='a', attr='e', idx=['PQ_12'])

array([0.])

In [8]:
ss.TDS.run()

Initialization for dynamics completed in 0.0181 seconds.
Initialization was successful.


  0%|          | 0/100 [00:00<?, ?%/s]

Simulation to t=20.00 sec completed in 0.2641 seconds.


True

Turned off devices will not inject power in TDS.

In [9]:
ss.Line.get(src='a1', attr='e', idx=['Line_21'])

array([0.])

## Scenario 2

Between successful PFlow solve and TDS initialization, turn off bus(es).

We load a case but do not set it up.

In [10]:
ss = andes.load(andes.get_case('ieee14/ieee14_conn.xlsx'),
                setup=False, no_output=True, default_config=True)

Working directory: "/Users/jinningwang/work/andes/examples/demonstration"
> Reloaded generated Python code of module "pycode".
Generated code for <PQ> is stale.
Numerical code generation (rapid incremental mode) started...


Generating code for 1 models on 12 processes.


Saved generated pycode to "/Users/jinningwang/.andes/pycode"
> Reloaded generated Python code of module "pycode".
Generated numerical code for 1 models in 0.0971 seconds.
Parsing input file "/Users/jinningwang/work/andes/andes/cases/ieee14/ieee14_conn.xlsx"...
Input file parsed in 0.0676 seconds.


First, let's turn on the bus manually.

> Note: it will ***not*** touch the connectivity manager to update bus online status before system setup.

In [11]:
ss.Bus.set(src='u', attr='v', idx=15, value=1)

In [12]:
ss.setup()

System internal structure set up in 0.0202 seconds.


True

In [13]:
ss.PFlow.run()

-> System connectivity check results:
  No islanded bus detected.
  System is interconnected.
  Each island has a slack bus correctly defined and enabled.

-> Power flow calculation
           Numba: Off
   Sparse solver: KLU
 Solution method: NR method
Power flow initialized in 0.0179 seconds.
0: |F(x)| = 0.7800160444
1: |F(x)| = 0.03157368515
2: |F(x)| = 4.343734372e-05
3: |F(x)| = 8.270892304e-11
Converged in 4 iterations in 0.0045 seconds.


True

Next, let's make a change on the parameter `Bus.name`.

In [14]:
ss.Bus.set(src='name', attr='v', idx=15, value='BUSTEST')

Such a change will not set the connectivity action flag.

In [15]:
ss.conn.is_needed

False

In contrast, if changes are made to `Bus.u.v`, the connectivity action flag is set.

In [16]:
ss.Bus.set(src='u', attr='v', idx=15, value=0)
ss.conn.is_needed

Bus turned off: [15]
Bus connectivity is touched, resolve PFlow before running EIG or TDS!


True

Remember to run the `PFlow` again if updates are made to the bus online status,
since this will reset the `PFlow` convergence flag.

In [17]:
ss.PFlow.converged

False

In [18]:
ss.PFlow.run()

-> System connectivity check results:
  No islanded bus detected.
  System is interconnected.
  Each island has a slack bus correctly defined and enabled.

-> Power flow calculation
           Numba: Off
   Sparse solver: KLU
 Solution method: NR method
Entering connectivity update.
Following bus(es) are turned off: [15]
-> System connectivity update results:
In <ACLine>, turn off ['Line_21']
In <StaticGen>, turn off [6]
In <StaticLoad>, turn off ['PQ_12']
In <StaticShunt>, turn off ['Shunt_3']
-> System connectivity check results:
  1 islanded bus detected.
  System is interconnected.
  Each island has a slack bus correctly defined and enabled.
Power flow initialized in 0.0051 seconds.
0: |F(x)| = 0.5605182134
1: |F(x)| = 0.006202200332
2: |F(x)| = 5.819382825e-06
3: |F(x)| = 6.964193111e-12
Converged in 4 iterations in 0.0096 seconds.


True

In [19]:
ss.EIG.run()

Initialization for dynamics completed in 0.0208 seconds.
Initialization was successful.

-> Eigenvalue Analysis:
4 states are associated with zero time constants. 
  Positive       0
  Zeros          1
  Negative      62
Eigenvalue analysis finished in 0.0017 seconds.


True

In [20]:
ss.TDS.run()


-> Time Domain Simulation Summary:
Sparse Solver: KLU
Simulation time: 0.0-20.0 s.
Fixed step size: h=33.33 ms. Shrink if not converged.


  0%|          | 0/100 [00:00<?, ?%/s]

Simulation to t=20.00 sec completed in 0.2661 seconds.


True

> **NOTE:** Turn on a bus will not automatically turn on the connected devices.

There are two cases that are not supported yet:

1. It is not supported to turn on a bus after successfully setting up a system.
1. It is not supported to turn on or off a bus after TDS initialization.